In [1]:
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

In [2]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Laptop GPU'

In [3]:
from model.deepsdf import DeepSDFDecoder
from util.model import summarize_model
from training import train_deepsdf
from data.shape_implicit import ShapeImplicit
from util.visualization import visualize_mesh, visualize_pointcloud

In [4]:
from data.shape_implicit import ShapeImplicit

train_dataset = ShapeImplicit(4000, 'train')
val_dataset= ShapeImplicit(4000, 'val')
overfit_dataset = ShapeImplicit(4000, 'overfit')
print(len(train_dataset[0]['sdf']))


4000


In [5]:

deepsdf = DeepSDFDecoder(latent_size=256)
print(summarize_model(deepsdf))

# input to the network is a concatenation of point coordinates (3) and the latent code (256 in this example);
# here we use a batch of 4096 points
input_tensor = torch.randn(4096, 3 + 256)
predictions = deepsdf(input_tensor)

print('\nOutput tensor shape: ', predictions.shape)  # expected output: 4096, 1

num_trainable_params = sum(p.numel() for p in deepsdf.parameters() if p.requires_grad) / 1e6
print(f'\nNumber of traininable params: {num_trainable_params:.2f}M')  # expected output: ~1.8M

   | Name      | Type           | Params 
-----------------------------------------------
0  | layer1    | Sequential     | 790010 
1  | layer1.0  | Linear         | 133632 
2  | layer1.1  | Dropout        | 0      
3  | layer1.2  | ReLU           | 0      
4  | layer1.3  | Linear         | 263168 
5  | layer1.4  | Dropout        | 0      
6  | layer1.5  | ReLU           | 0      
7  | layer1.6  | Linear         | 263168 
8  | layer1.7  | Dropout        | 0      
9  | layer1.8  | ReLU           | 0      
10 | layer1.9  | Linear         | 130042 
11 | layer1.10 | Dropout        | 0      
12 | layer1.11 | ReLU           | 0      
13 | layer2    | Sequential     | 1053186
14 | layer2.0  | Linear         | 263168 
15 | layer2.1  | Dropout        | 0      
16 | layer2.2  | ReLU           | 0      
17 | layer2.3  | Linear         | 263168 
18 | layer2.4  | Dropout        | 0      
19 | layer2.5  | ReLU           | 0      
20 | layer2.6  | Linear         | 263168 
21 | layer2.7  | Dropout    

In [5]:


overfit_config = {
    'experiment_name': '3_2_deepsdf_overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'num_sample_points': 40000,
    'latent_code_length': 256,
    'batch_size': 16,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,
    'print_every_n': 50,
    'visualize_every_n': 250,
}

train_deepsdf.main(overfit_config)  # expected loss around 0.0062

Using device: cuda:0
[049/00000] train_loss: 0.038085
[099/00000] train_loss: 0.024551
[149/00000] train_loss: 0.017208
[199/00000] train_loss: 0.013007
[249/00000] train_loss: 0.011381
[299/00000] train_loss: 0.010379
[349/00000] train_loss: 0.009594
[399/00000] train_loss: 0.009046
[449/00000] train_loss: 0.008606
[499/00000] train_loss: 0.008203
[549/00000] train_loss: 0.007759
[599/00000] train_loss: 0.007510
[649/00000] train_loss: 0.007367
[699/00000] train_loss: 0.007235
[749/00000] train_loss: 0.007095
[799/00000] train_loss: 0.006966
[849/00000] train_loss: 0.006858
[899/00000] train_loss: 0.006739
[949/00000] train_loss: 0.006704
[999/00000] train_loss: 0.006567
[1049/00000] train_loss: 0.006402
[1099/00000] train_loss: 0.006352
[1149/00000] train_loss: 0.006297
[1199/00000] train_loss: 0.006250
[1249/00000] train_loss: 0.006198
[1299/00000] train_loss: 0.006165
[1349/00000] train_loss: 0.006095
[1399/00000] train_loss: 0.006054
[1449/00000] train_loss: 0.006010
[1499/00000] 

In [6]:
# Load and visualize GT mesh of the overfit sample
gt_mesh = ShapeImplicit.get_mesh('7e728818848f191bee7d178666aae23d')
print('GT')
visualize_mesh(gt_mesh.vertices, gt_mesh.faces, flip_axes=True)

# Load and visualize reconstructed overfit sample; it's okay if they don't look visually exact, since we don't run 
# the training too long and have a learning rate decay while training 
mesh_path = "runs/3_2_deepsdf_overfit/meshes/01999_000.obj"
overfit_output = trimesh.load(mesh_path)
print('Overfit')
visualize_mesh(overfit_output.vertices, overfit_output.faces, flip_axes=True)

GT


C:\Users\baris\anaconda3\envs\adl4cv\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

Overfit


Output()

In [6]:
from training import train_deepsdf

generalization_config = {
    'experiment_name': '3_2_deepsdf_generalization',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'num_sample_points': 4096, # you can adjust this such that the model fits on your gpu
    'latent_code_length': 256,
    'batch_size': 1,
    'resume_ckpt': None,
    'learning_rate_model': 0.0005,
    'learning_rate_code': 0.001,
    'lambda_code_regularization': 0.0001,
    'max_epochs': 2000,  # not necessary to run for 2000 epochs if you're short on time, at 500 epochs you should start to see reasonable results
    'print_every_n': 50,
    'visualize_every_n': 5000,
}

train_deepsdf.main(generalization_config)

Using device: cuda:0
[000/00049] train_loss: 0.038442
[000/00099] train_loss: 0.035935
[000/00149] train_loss: 0.033375
[000/00199] train_loss: 0.033340
[000/00249] train_loss: 0.032919
[000/00299] train_loss: 0.032582
[000/00349] train_loss: 0.033365
[000/00399] train_loss: 0.031029
[000/00449] train_loss: 0.032137
[000/00499] train_loss: 0.030671
[000/00549] train_loss: 0.031336
[000/00599] train_loss: 0.032829
[000/00649] train_loss: 0.031796
[000/00699] train_loss: 0.032844
[000/00749] train_loss: 0.031459
[000/00799] train_loss: 0.031331
[000/00849] train_loss: 0.032159
[000/00899] train_loss: 0.030964
[000/00949] train_loss: 0.032007
[000/00999] train_loss: 0.033122
[000/01049] train_loss: 0.031459
[000/01099] train_loss: 0.031270
[000/01149] train_loss: 0.032139
[000/01199] train_loss: 0.031048
[001/00023] train_loss: 0.031830
[001/00073] train_loss: 0.032459
[001/00123] train_loss: 0.031237
[001/00173] train_loss: 0.030809
[001/00223] train_loss: 0.030694
[001/00273] train_loss

KeyboardInterrupt: 

In [4]:
from inference.infer_deepsdf import InferenceHandlerDeepSDF

device = torch.device('cuda:0') 

inference_handler = InferenceHandlerDeepSDF(256, "runs/3_2_deepsdf_generalization", device)

In [5]:
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are very complete
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Observations with negative SDF (inside)


Output()

Observations with positive SDF (outside)


Output()

In [9]:
from inference.infer_deepsdf import InferenceHandlerDeepSDF

device = torch.device('cuda:0') 

inference_handler = InferenceHandlerDeepSDF(256, "runs/3_2_deepsdf_generalization", device)
vertices, faces,vector = inference_handler.infer_from_latent_code(5)
print(vector)
visualize_mesh(vertices, faces, flip_axes=True)

tensor([ 3.9678e-03,  1.1214e-01, -4.9493e-02,  7.5664e-02,  6.9372e-02,
        -1.7136e-02,  1.0058e-02,  4.1867e-02,  9.4704e-02,  5.0239e-02,
         1.7597e-01,  2.9160e-02, -8.6489e-02, -1.1565e-01,  5.1267e-02,
         3.7934e-02,  3.7827e-02,  4.0937e-02,  1.2031e-01, -3.3028e-02,
        -4.1881e-02,  2.3247e-02,  1.1089e-01,  3.6403e-02,  1.0604e-01,
        -8.8085e-02,  8.7894e-02, -1.6806e-02, -3.9747e-02, -2.3614e-02,
         2.2089e-02,  2.0699e-02, -8.9851e-04,  1.1043e-02,  5.2559e-03,
        -1.0075e-01, -5.9560e-02,  7.6523e-02,  1.9515e-02, -6.3301e-02,
         8.8997e-02, -3.8780e-02, -2.9116e-02,  6.6308e-02,  1.4697e-02,
        -1.3006e-01, -4.3984e-03,  7.3843e-02, -4.8756e-02,  1.2220e-03,
         9.1506e-02,  2.2524e-02,  5.2360e-02, -7.1146e-02,  5.5575e-02,
        -6.2527e-02, -6.3008e-03,  9.1550e-02, -1.0216e-01,  6.7139e-02,
        -9.1633e-02, -5.6084e-02, -3.0409e-02,  2.8497e-03, -3.0263e-03,
        -7.1003e-02,  1.1182e-02, -6.6428e-02, -8.1

Output()

In [10]:
change_index = [0,10,100,250]
change_value = [1,1,1,1]

vertices, faces, sdf = inference_handler.infer_from_latent_code_modify(5 ,change_index, change_value)
print(sdf)
visualize_mesh(vertices, faces, flip_axes=True)

tensor([ 1.0000e+00,  1.1214e-01, -4.9493e-02,  7.5664e-02,  6.9372e-02,
        -1.7136e-02,  1.0058e-02,  4.1867e-02,  9.4704e-02,  5.0239e-02,
         1.0000e+00,  2.9160e-02, -8.6489e-02, -1.1565e-01,  5.1267e-02,
         3.7934e-02,  3.7827e-02,  4.0937e-02,  1.2031e-01, -3.3028e-02,
        -4.1881e-02,  2.3247e-02,  1.1089e-01,  3.6403e-02,  1.0604e-01,
        -8.8085e-02,  8.7894e-02, -1.6806e-02, -3.9747e-02, -2.3614e-02,
         2.2089e-02,  2.0699e-02, -8.9851e-04,  1.1043e-02,  5.2559e-03,
        -1.0075e-01, -5.9560e-02,  7.6523e-02,  1.9515e-02, -6.3301e-02,
         8.8997e-02, -3.8780e-02, -2.9116e-02,  6.6308e-02,  1.4697e-02,
        -1.3006e-01, -4.3984e-03,  7.3843e-02, -4.8756e-02,  1.2220e-03,
         9.1506e-02,  2.2524e-02,  5.2360e-02, -7.1146e-02,  5.5575e-02,
        -6.2527e-02, -6.3008e-03,  9.1550e-02, -1.0216e-01,  6.7139e-02,
        -9.1633e-02, -5.6084e-02, -3.0409e-02,  2.8497e-03, -3.0263e-03,
        -7.1003e-02,  1.1182e-02, -6.6428e-02, -8.1

Output()

In [14]:
vertices, faces, sdf = inference_handler.infer_from_latent_code_random(100)
#visualize_mesh(vertices, faces, flip_axes=True)

In [33]:
print(torch.randn(256 ))

tensor([ 1.4512, -0.2293,  1.6011,  0.5525, -0.0056,  0.1566,  0.6482,  0.9372,
         0.1635,  0.4690,  0.6803, -1.5080,  0.5878,  0.9822,  0.0767, -0.4377,
        -0.1356,  0.8706, -0.0089,  0.8198, -0.3117,  1.0769, -0.2785, -0.7350,
         0.1414, -1.3675,  0.1254, -0.4071,  1.3408, -1.7143,  0.1022,  0.2672,
        -1.5632, -0.0304,  0.0187,  0.7728, -1.2126, -0.4962, -0.5095, -0.1079,
        -1.8425, -0.5112, -0.9941,  0.6902, -0.6030, -1.2719,  1.1134, -0.2714,
        -0.5084, -2.2681, -0.9738, -0.8521,  0.0044,  0.0542, -0.4011,  0.1988,
         0.6645, -0.0876,  0.1539,  0.4829, -0.2615, -2.4983,  2.5437,  0.0434,
         0.2490, -0.7489,  0.6086,  0.6153, -0.9611,  1.5126, -0.4777, -2.3318,
        -0.1743,  0.8943, -0.5295,  0.3597,  0.0536, -0.9545,  0.1996, -0.3900,
         0.2704, -1.4886,  0.2606,  0.3225, -0.8994,  1.2399, -0.4724,  0.3016,
         0.5105,  1.1242, -0.1556,  0.2045, -0.2188,  0.8622, -0.7514,  1.2846,
        -0.4418, -0.3430,  0.8116,  0.74

In [6]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

[00000] optim_loss: 0.032907
[00050] optim_loss: 0.007684
[00100] optim_loss: 0.006094
[00150] optim_loss: 0.005828
[00200] optim_loss: 0.005806
[00250] optim_loss: 0.005244
[00300] optim_loss: 0.005009
[00350] optim_loss: 0.004888
[00400] optim_loss: 0.004799
[00450] optim_loss: 0.004727
[00500] optim_loss: 0.004729
[00550] optim_loss: 0.004672
[00600] optim_loss: 0.004632
[00650] optim_loss: 0.004426
[00700] optim_loss: 0.004569
[00750] optim_loss: 0.004350
Optimization complete.


Output()

In [7]:
# get observed data
points, sdf = ShapeImplicit.get_all_sdf_samples("b351e06f5826444c19fb4103277a6b93_incomplete")

inside_points = points[sdf[:, 0] < 0, :].numpy()
outside_points = points[sdf[:, 0] > 0, :].numpy()

# visualize observed points; you'll observe that the observations are incomplete
# making this is a shape completion task
print('Observations with negative SDF (inside)')
visualize_pointcloud(inside_points, 0.025, flip_axes=True)
print('Observations with positive SDF (outside)')
visualize_pointcloud(outside_points, 0.025, flip_axes=True)

Observations with negative SDF (inside)


Output()

Observations with positive SDF (outside)


Output()

In [8]:
# reconstruct
vertices, faces = inference_handler.reconstruct(points, sdf, 800)
# visualize
visualize_mesh(vertices, faces, flip_axes=True)

[00000] optim_loss: 0.033109
[00050] optim_loss: 0.007933
[00100] optim_loss: 0.006220
[00150] optim_loss: 0.006176
[00200] optim_loss: 0.005622
[00250] optim_loss: 0.005707
[00300] optim_loss: 0.005166
[00350] optim_loss: 0.005123
[00400] optim_loss: 0.004846
[00450] optim_loss: 0.004587
[00500] optim_loss: 0.004600
[00550] optim_loss: 0.004557
[00600] optim_loss: 0.004522
[00650] optim_loss: 0.004315
[00700] optim_loss: 0.004458
[00750] optim_loss: 0.004181
Optimization complete.


Output()

In [9]:
from data.shape_implicit import ShapeImplicit
from util.visualization import visualize_mesh

mesh = ShapeImplicit.get_mesh("494fe53da65650b8c358765b76c296")
print('GT Shape A')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

mesh = ShapeImplicit.get_mesh("5ca1ef55ff5f68501921e7a85cf9da35")
print('GT Shape B')
visualize_mesh(mesh.vertices, mesh.faces, flip_axes=True)

GT Shape A


Output()

GT Shape B


Output()

In [10]:
from inference.infer_deepsdf import InferenceHandlerDeepSDF

inference_handler = InferenceHandlerDeepSDF(256, "runs/3_2_deepsdf_generalization", torch.device('cuda:0'))
# interpolate; also exports interpolated meshes to disk
inference_handler.interpolate('494fe53da65650b8c358765b76c296', '5ca1ef55ff5f68501921e7a85cf9da35', 60)

In [14]:
from util.mesh_collection_to_gif import  meshes_to_gif
from util.misc import show_gif

# create list of meshes (just exported) to be visualized
mesh_paths = sorted([x for x in Path("/runs/3_2_deepsdf_generalization/interpolation").iterdir() if int(x.name.split('.')[0].split("_")[1]) == 0], key=lambda x: int(x.name.split('.')[0].split("_")[0]))
mesh_paths = mesh_paths + mesh_paths[::-1]

# create a visualization of the interpolation process
meshes_to_gif(mesh_paths, "exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif", 20)
show_gif("exercise_3/runs/3_2_deepsdf_generalization/latent_interp.gif")

ModuleNotFoundError: No module named 'pyrender'